In [4]:
import numpy as np
import os
import json
import csv
import pandas as pd

base_score_dir = '/home/ubuntu/MMSci/mmsci-exps/eval/eval_scores/image_caption_generation/textgen'
target_score_dir = '/home/ubuntu/MMSci/mmsci-exps/eval/eval_scores/image_caption_generation'
csv_dir = os.path.join(target_score_dir, 'csv')

In [5]:
model_list = ['blip2', 'kosmos2', 'qwen', 'llava', 'llava-next', 'llava-next-mistral', 'gpt-4-turbo', 'gpt-4o', 'llava-next-mmsci']
metric_list = ['BLEU-1', 'BLEU-2', 'BLEU-3', 'BLEU-4', 'BLEU', 'METEOR', 'ROUGE', 'ROUGE1', 'ROUGE2', 'ROUGEL', 'rougeLSUM', 'CIDER', 'CLIPScore', 'RefCLIPScore']
fields = ['model']
for metric in metric_list:
    fields.extend([f'{metric} [mean]', f'{metric} [std]'])

for w_abs in [False, True]:
    for w_ctx in [False, True]:
        if w_abs and w_ctx:
            continue
        tag = f'abs{w_abs}_ctx{w_ctx}'

        with open(os.path.join(csv_dir, f'{tag}.csv'), 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fields)
            writer.writeheader()
            score_dict = []

            for model in model_list:
                filename = os.path.join(base_score_dir, tag, f'{model}.json')
                if not os.path.exists(filename):
                    continue
                all_scores = json.load(open(filename))
                info = {'model': model}
                for metric in metric_list:
                    if metric in all_scores:
                        scores = all_scores[metric]
                        mean = np.mean(scores)
                        std = np.std(scores)
                        info[f'{metric} [mean]'] = mean
                        info[f'{metric} [std]'] = std
                    else:
                        info[f'{metric} [mean]'] = 0.0
                        info[f'{metric} [std]'] = 0.0
                score_dict.append(info)

            writer.writerows(score_dict)



In [ ]:
w_abs = False
w_ctx = False

tag = f'abs{w_abs}_ctx{w_ctx}'

# Load the CSV file
cap_score_path = os.path.join(target_score_dir, "csv", f'{tag}.csv')
df = pd.read_csv(cap_score_path)
print(df.columns)

In [ ]:
# Filter columns that contain "[mean]"
filtered_columns = [col for col in df.columns if "[mean]" in col or "model" in col]

# Check if the string is in the 'Name' column (case-sensitive)
filtered_df = df[filtered_columns]

numeric_cols = filtered_df.select_dtypes(include='number').columns
filtered_df[numeric_cols] = filtered_df[numeric_cols].applymap(lambda x: x * 100)
filtered_df[numeric_cols] = filtered_df[numeric_cols].round(2)

# Print the filtered DataFrame
print(filtered_df)